# CloudWatch Alerting System Setup

<div style="background-color: #f8f9fa; border: 1px solid #e9ecef; border-radius: 8px; padding: 10px; margin: 10px;">
<strong>📋 Workshop Contents</strong>
<ul style="line-height: 1.2;">
<li><a href="#Prerequisites">Prerequisites</a></li>
<li><a href="#Setting-up-the-environment-and-required-libraries">Setting up the environment and required libraries</a></li>
<li><a href="#1-Setting-up-SNS-Topic-for-Notifications">1. Setting up SNS Topic for Notifications</a></li>
<li><a href="#2-Configuring-CloudWatch-Alarms">2. Configuring CloudWatch Alarms</a></li>
<li><a href="#3-Creating-Custom-Alert-Thresholds">3. Creating Custom Alert Thresholds</a></li>
<li><a href="#Usage-Example">Usage Example</a></li>
</ul>
</div>

## Prerequisites

Before starting this workshop, ensure you have:

- ✅ **Jupyter Notebook**: You can launch a [free tier Amazon SageMaker Jupyter Notebook](../../1_Getting_Started_with_AWS/1.4_Setting_up_Your_Cookbook_Environment/README.MD)
- ✅ **Aurora PostgreSQL Cluster**: An active Aurora PostgreSQL cluster running in your AWS account
  - If you don't have one, follow the setup guide: [Your First Database on AWS](../../2_Your_First_Database_on_AWS/README.MD)
- ✅ **AWS CLI configured** with appropriate permissions for CloudWatch and SNS
- ✅ **IAM permissions** for:
  - CloudWatch alarms and metrics
  - SNS topic creation and management
  - Email subscription confirmations

> 💡 **Note**: You'll need to confirm email subscriptions to receive alert notifications.


## 1. Setting up SNS Topic for Notifications

Create and configure SNS topics for alert notifications to enable automated email alerts when CloudWatch alarms trigger. The create_sns_topic function establishes the notification channel, while subscribe_to_sns_topic adds email subscribers to receive alerts with topic ARN and subscription confirmation responses.

In [ ]:
%%bash

# Set your variables
TOPIC_NAME="your-topic-name"
EMAIL="your-email@example.com"

# Create SNS topic
if TOPIC_ARN=$(aws sns create-topic --name ${TOPIC_NAME} --tags Key=CreationSource,Value=aws-database-cookbook-v2025.8 --output text --query 'TopicArn'); then
    echo "Successfully created SNS topic: ${TOPIC_ARN}"
else
    echo "Failed to create SNS topic"
    exit 1
fi

# Subscribe email to the topic
if aws sns subscribe \
    --topic-arn ${TOPIC_ARN} \
    --protocol email \
    --notification-endpoint ${EMAIL}; then
    echo "Successfully subscribed ${EMAIL} to the topic"
else
    echo "Failed to subscribe to the topic"
    exit 1
fi

## 2. Configuring CloudWatch Alarms

Create CloudWatch metric alarms with customizable thresholds and comparison operators for comprehensive monitoring. The alarm configuration includes evaluation periods, statistics, and actions to trigger when thresholds are breached with alarm creation response showing metric criteria and notification settings.

In [ ]:
%%bash

# Set your variables
METRIC_NAME="your-metric-name"
NAMESPACE="your-namespace"
THRESHOLD="your-threshold-value"
COMPARISON_OPERATOR="GreaterThanThreshold"  # Or other operators like LessThanThreshold
TOPIC_ARN="your-sns-topic-arn"
DIMENSION_NAME="DBInstanceIdentifier"  # Example dimension name
DIMENSION_VALUE="your-instance-id"     # Example dimension value

# Create CloudWatch Alarm
aws cloudwatch put-metric-alarm \
    --alarm-name "${METRIC_NAME}-Alarm" \
    --comparison-operator ${COMPARISON_OPERATOR} \
    --evaluation-periods 2 \
    --metric-name ${METRIC_NAME} \
    --namespace ${NAMESPACE} \
    --period 300 \
    --statistic Average \
    --threshold ${THRESHOLD} \
    --actions-enabled \
    --alarm-actions ${TOPIC_ARN} \
    --alarm-description "Alarm for ${METRIC_NAME}" \
    --dimensions "Name=${DIMENSION_NAME},Value=${DIMENSION_VALUE}" \
    --tags Key=CreationSource,Value=aws-database-cookbook-v2025.8

## 3. Creating Custom Alert Thresholds

Create composite alarms that combine multiple metric conditions using logical operators for sophisticated alerting scenarios. Composite alarms enable complex alert logic by evaluating multiple individual alarms together using AND/OR conditions with composite alarm creation response showing alarm rule logic.

In [ ]:
%%bash

# Set your variables
ALARM_NAME="your-composite-alarm-name"
ALARM_RULE="ALARM(metric1-Alarm) OR ALARM(metric2-Alarm)"
TOPIC_ARN="your-sns-topic-arn"

# Create composite alarm with error checking
if aws cloudwatch put-composite-alarm \
    --alarm-name "${ALARM_NAME}" \
    --alarm-rule "${ALARM_RULE}" \
    --actions-enabled \
    --alarm-actions "${TOPIC_ARN}" \
    --alarm-description "Custom composite alarm: ${ALARM_NAME}" \
    --tags Key=CreationSource,Value=aws-database-cookbook-v2025.8; then
    echo "Successfully created composite alarm: ${ALARM_NAME}"
else
    echo "Failed to create composite alarm"
    exit 1
fi

## Usage Example

Demonstrate setting up a complete alerting system with SNS notifications, individual metric alarms, and composite alarms for advanced monitoring scenarios. The script creates email notifications, configures CPU and memory alerts, and establishes composite alarm logic with confirmation messages for each component.

In [ ]:
%%bash

# Exit on error
set -e

# Configuration variables
TOPIC_NAME="monitoring-alerts"
EMAIL="your-email@example.com"

# Create SNS topic
echo "Creating SNS topic..."
if ! TOPIC_ARN=$(aws sns create-topic --name ${TOPIC_NAME} --tags Key=CreationSource,Value=aws-database-cookbook-v2025.8 --output text --query 'TopicArn'); then
    echo "Failed to create SNS topic"
    exit 1
fi
echo "Created SNS Topic: ${TOPIC_ARN}"

# Subscribe email
echo "Subscribing email to SNS topic..."
if ! aws sns subscribe \
    --topic-arn ${TOPIC_ARN} \
    --protocol email \
    --notification-endpoint ${EMAIL}; then
    echo "Failed to subscribe email to topic"
    exit 1
fi

# Define alerts
read -r -d '' ALERTS << EOM
[
  {
    "metric_name": "CPUUtilization",
    "namespace": "AWS/EC2",
    "threshold": 80,
    "comparison_operator": "GreaterThanThreshold",
    "dimensions": [{"Name": "InstanceId", "Value": "i-1234567890abcdef0"}]
  },
  {
    "metric_name": "MemoryUtilization",
    "namespace": "CustomMetrics",
    "threshold": 90,
    "comparison_operator": "GreaterThanThreshold",
    "dimensions": [{"Name": "ServiceName", "Value": "MyApp"}]
  }
]
EOM

# Create metric alarms
echo "Creating metric alarms..."
echo "${ALERTS}" | jq -c '.[]' | while read -r alert; do
    METRIC_NAME=$(echo "${alert}" | jq -r '.metric_name')
    NAMESPACE=$(echo "${alert}" | jq -r '.namespace')
    THRESHOLD=$(echo "${alert}" | jq -r '.threshold')
    COMPARISON_OPERATOR=$(echo "${alert}" | jq -r '.comparison_operator')
    DIMENSIONS=$(echo "${alert}" | jq -r '.dimensions')

    if ! aws cloudwatch put-metric-alarm \
        --alarm-name "${METRIC_NAME}-Alarm" \
        --comparison-operator ${COMPARISON_OPERATOR} \
        --evaluation-periods 2 \
        --metric-name ${METRIC_NAME} \
        --namespace ${NAMESPACE} \
        --period 300 \
        --statistic Average \
        --threshold ${THRESHOLD} \
        --actions-enabled \
        --alarm-actions ${TOPIC_ARN} \
        --alarm-description "Alarm for ${METRIC_NAME}" \
        --dimensions "${DIMENSIONS}" \
        --tags Key=CreationSource,Value=aws-database-cookbook-v2025.8; then
        echo "Failed to create alarm for ${METRIC_NAME}"
        exit 1
    fi
    echo "Created alarm for ${METRIC_NAME}"
done

# Create composite alarm
echo "Creating composite alarm..."
if ! aws cloudwatch put-composite-alarm \
    --alarm-name "HighLoadCompositeAlarm" \
    --alarm-rule "ALARM(CPUUtilization-Alarm) AND ALARM(MemoryUtilization-Alarm)" \
    --actions-enabled \
    --alarm-actions "${TOPIC_ARN}" \
    --alarm-description "Custom composite alarm: HighLoadCompositeAlarm" \
    --tags Key=CreationSource,Value=aws-database-cookbook-v2025.8; then
    echo "Failed to create composite alarm"
    exit 1
fi

echo "Setup complete!"

## Next Steps 🚀

Now that you've set up comprehensive alerting for your Aurora cluster, continue building operational excellence with these next steps:

1. **Implement Automated Backup Solutions**
   - Set up automated backup strategies in [4.3 Automated Backup Solutions](../4.3_Automated_Backup_Solutions/automate_backup.ipynb)
   - Configure point-in-time recovery and cross-region backups

2. **Optimize Database Performance**
   - Use performance optimization tools in [4.4 Performance Optimization Tools](../4.4_Performance_Optimization_Tools/performance_optimization.ipynb)
   - Learn query optimization and resource tuning techniques

3. **Set Up Development and Testing Environments**
   - Implement Aurora cloning for testing in [4.5 Aurora Cloning for Testing and Development](../4.5_Aurora_Cloning_for_Testing_and_Development/aurora_cloning_testing_devlop.ipynb)
   - Create isolated environments for safe testing and development

Continue to [4.3 Implement Automated Backup Solutions](../4.3_Automated_Backup_Solutions/automate_backup.ipynb).


## Additional Resources 📚

### CloudWatch Alarms
- [CloudWatch Alarms Overview](https://docs.aws.amazon.com/AmazonCloudWatch/latest/monitoring/AlarmThatSendsEmail.html)
- [Composite Alarms](https://docs.aws.amazon.com/AmazonCloudWatch/latest/monitoring/Create_Composite_Alarm.html)
- [Alarm Actions](https://docs.aws.amazon.com/AmazonCloudWatch/latest/monitoring/AlarmThatSendsEmail.html#alarms-and-actions)

### SNS Notifications
- [Amazon SNS Getting Started](https://docs.aws.amazon.com/sns/latest/dg/sns-getting-started.html)
- [SNS Email Notifications](https://docs.aws.amazon.com/sns/latest/dg/sns-email-notifications.html)
- [SNS Best Practices](https://docs.aws.amazon.com/sns/latest/dg/sns-best-practices.html)

### Aurora Specific Monitoring
- [Aurora CloudWatch Metrics](https://docs.aws.amazon.com/AmazonRDS/latest/AuroraUserGuide/MonitoringAurora.html)
- [Aurora Alerting Best Practices](https://docs.aws.amazon.com/AmazonRDS/latest/AuroraUserGuide/Aurora.BestPractices.html)
- [Performance Insights Alarms](https://docs.aws.amazon.com/AmazonRDS/latest/AuroraUserGuide/USER_PerfInsights.html)